In [14]:
from modules.ranks import Ranks
from modules.prices import Prices

from datetime import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymssql

### Enter data

In [6]:
cash = 1000
equity = 1000000

model_file = "files/model4.joblib"
stocktrak_file = "files/OpenPosition_3_19_2022.csv"

# trade out of everything beyond threshold_rank
threshold_rank = 300

# target for positions
target_weight = 1/42

# max weight allowed, long or short
threshold_weight = 0.04

# trade when max weight exceeded 
#     if rank better than rebalance_rank: trade to target
#     if rank worse than rebalance_rank: trade to zero
rebalance_rank = 40

# don't trade if trade size would be less than minimum
minimum_investment = 0.01

# trade to target if rank better than rebalance_rank and weight below topup_weight
topup_weight = 0.01

### Update prices, positions, ranks, equity, and weights

In [ ]:
ranks = Ranks(model_file)

In [7]:
prices = Prices(stocktrak_file)

made connection
got SQL data


In [8]:
positions = pd.read_csv(stocktrak_file)
positions = positions[["Symbol", "Quantity"]]
positions.columns = ["ticker", "position"]
positions = positions.set_index("ticker")
positions = positions.squeeze()

positions = positions.reindex(ranks.index).fillna(0)
prices = prices.reindex(ranks.index)

equity = cash + (positions*prices).sum()
weights = (positions*prices) / equity

### Long side

In [11]:
sellall = ranks>threshold_rank
sellall = sellall  | ((weights>threshold_weight) & (ranks>rebalance_rank))
sellall = sellall & (positions>0)

sellsome = (weights>threshold_weight) & (ranks<=rebalance_rank)
sellsome = sellsome & (positions>0)
sellsomeamounts = (equity*(weights-target_weight)/prices).astype(int)

trades = - np.where(sellall, positions, 0)
trades -= np.where(sellsome, sellsomeamounts, 0)
trades = pd.Series(trades, index=ranks.index)

temp = positions + trades
longs = (temp*prices)[temp>0].sum()
tickers = ranks.sort_values().index.to_list()

if longs < equity:
    toinvest = equity - longs
    while toinvest >= minimum_investment*equity:
        tick = tickers.pop(0)
        if positions[tick] == 0:
            dollars = min(toinvest, target_weight*equity)
            trades[tick] = int(dollars/prices[tick])
            toinvest -= trades[tick]*prices[tick]
        elif weights[tick] < topup_weight:
            dollars = min(cash, (target_weight-weights[tick])*equity)
            trades[tick] = int(dollars/prices[tick])
            toinvest -= trades[tick]*prices[tick]
else :
    tosell = longs - equity
    while tosell >= minimum_investment*equity:
        tick = tickers.pop()
        if positions[tick] > 0:
            trades[tick] = - positions[tick]
            tosell += trades[tick]*prices[tick]

### Short side

In [12]:
shortranks = ranks.max() - ranks
shortweights = - weights

coverall = shortranks>threshold_rank
coverall = coverall  | ((shortweights>threshold_weight) & (shortranks>rebalance_rank))
coverall = coverall & (positions<0)

coversome = (shortweights>threshold_weight) & (shortranks<=rebalance_rank)
coversome = coversome & (positions<0)
coversomeamounts = (equity*(shortweights-target_weight)/prices).astype(int)

trades = trades.to_numpy()
trades -= np.where(coverall, positions, 0)
trades -= np.where(coversome, coversomeamounts, 0)
trades = pd.Series(trades, index=ranks.index)

temp = positions + trades
shorts = -(temp*prices)[temp<0].sum()
tickers = ranks.sort_values().index.to_list()

if shorts < equity:
    toshort = equity - shorts
    while toshort >= minimum_investment*equity:
        tick = tickers.pop(0)
        if positions[tick] == 0:
            dollars = min(toshort, target_weight*equity)
            trades[tick] = -int(dollars/prices[tick])
            toshort += trades[tick]*prices[tick]
        elif shortweights[tick] < topup_weight:
            dollars = min(toshort, (target_weight-shortweights[tick])*equity)
            trades[tick] = -int(dollars/prices[tick])
            toshort += trades[tick]*prices[tick]
else :
    tocover = shorts - equity
    while tocover >= minimum_investment*equity:
        tick = tickers.pop()
        if positions[tick] < 0:
            trades[tick] = - positions[tick]
            tocover -= trades[tick]*prices[tick]



In [35]:
output = pd.concat((ranks, positions, trades, prices), axis=1)
output.columns = ["rnk", "position", "trade", "price"]

server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

info = pd.read_sql("select * from tickers", conn).set_index("ticker")
info = info[~info.index.duplicated()]
output = output.join(info, how="left")
output = output.sort_values(by="rnk")

today = datetime.today()
today = today.strftime("%Y-%m-%d")
output.to_csv("files/output-" + today + ".csv")